In [35]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from catboost import CatBoostRegressor, Pool
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import cross_val_score
from datetime import datetime, date
import joblib

In [5]:
def mape(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred) / y_true * 100)

mape_score = make_scorer(mape, greater_is_better=False)

In [6]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.shape

(51815, 43)

In [7]:
train = train[train.price != 0]
train.shape

(51775, 43)

In [9]:
train_X, valid_X = train_test_split(train, test_size=0.3, random_state=42)
#train_X = train

In [10]:
train_X.iloc[:5, :10]

,id,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access
21300,21188508,BIG DOUBLE ROOM WITH 2 bed.,Beautiful. Must be occupied by TWO guests at a...,The accommodation is the heart of the Town Cen...,Beautiful. Must be occupied by TWO guests at a...,none,The area is located in a friendly neighbourhoo...,The accommodation is located in a friendly env...,"There is 24 hours Bus Services to the City, Wi...","The areas accessible to guests are: Room, Kitc..."
8742,17230572,Croham Park Bed & Breakfast - Room 1,Room 1 (Hot Breakfast Selection Included) - Sp...,Welcome home! A boutique Bed & Breakfast set i...,Room 1 (Hot Breakfast Selection Included) - Sp...,none,"Quiet residential street. Well-lit street, if...",All room rates include traditional hot breakfa...,Getting to us is easy: By Car - Please simply ...,In the morning breakfast is served in the dini...
41079,27242002,Cosy double bedroom near the Thames in Fulham,A cosy bedroom in a modern 3 bedroom apartmen...,It's riverside and beautifully located next to...,A cosy bedroom in a modern 3 bedroom apartmen...,none,"The location is convenient, situated in the he...",There's no living room.,5 minutes walk from Putney Bridge Station. 7 ...,You will have your private bedroom and probabl...
45075,28197036,Luxury Studio Apartment,It is a studio shared room with a shared kitch...,NaN,It is a studio shared room with a shared kitch...,none,NaN,NaN,NaN,NaN
32280,24388059,★RARE FIND★ Unique Family House In Wimbledon P...,A superb three bedroom house in Wimbledon Park...,★What To Expect★ * Two King size and one doubl...,A superb three bedroom house in Wimbledon Park...,none,It’s impossible to write a guide to Wimbledon ...,• Do you have free wifi? Yes. We will give you...,Wimbledon Park station is an 8 minute walk fro...,The entire property will be yours to enjoy


In [11]:
train_X.iloc[:5, 10:19]

,interaction,house_rules,host_id,host_since,host_about,host_response_time,host_response_rate,host_is_superhost,host_has_profile_pic
21300,I am available 24/7 and can be contacted at an...,NaN,150313033,2017-09-12,"A lovely, caring woman who has a passion for s...",within a few hours,100%,f,t
8742,We like to great our guests and make them feel...,NaN,116054029,2017-02-12,NaN,NaN,NaN,f,t
41079,"As said above, we are 3 friends living togethe...",- Quiet after 11pm - Shared areas to keep clean,1419474,2011-11-18,NaN,within a few hours,100%,f,t
45075,NaN,Guests are not allowed to smoke or drink insid...,210342763,2018-08-18,NaN,NaN,NaN,f,t
32280,The management is done by a lovely local team ...,- - Noise must be kept to an absolute minimum ...,147514538,2017-08-24,"Hey Airbnb'ers!\r\nI'm Emily, I work for The A...",within an hour,98%,f,t


In [12]:
train_X.iloc[:5, 19:29]

,host_identity_verified,neighbourhood_cleansed,zipcode,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms
21300,f,Merton,CR4 2PE,51.406751,-0.162524,t,Apartment,Private room,2,1.0
8742,f,Croydon,CR2,51.360689,-0.082333,t,Bed and breakfast,Private room,2,1.0
41079,f,Hammersmith and Fulham,SW6 5JH,51.472258,-0.209634,t,Apartment,Private room,2,1.0
45075,f,Kingston upon Thames,KT1,51.409543,-0.288106,t,Apartment,Shared room,1,1.0
32280,f,Merton,NaN,51.439205,-0.199344,t,House,Entire home/apt,6,2.0


In [13]:
train_X.iloc[:5, 29:38]

,bedrooms,beds,bed_type,amenities,square_feet,security_deposit,cleaning_fee,guests_included,extra_people
21300,1.0,2.0,Real Bed,"{Wifi,""Air conditioning"",Kitchen,Heating,Washe...",NaN,0.0,20.0,1,15.0
8742,1.0,1.0,Real Bed,"{TV,Wifi,""Free parking on premises"",Breakfast,...",NaN,NaN,NaN,1,0.0
41079,1.0,1.0,Real Bed,"{Wifi,Kitchen,""Paid parking off premises"",Brea...",NaN,0.0,5.0,1,5.0
45075,1.0,1.0,Real Bed,"{TV,Wifi,Kitchen,Elevator,""Free street parking...",NaN,0.0,12.0,1,0.0
32280,3.0,3.0,Real Bed,"{Internet,Wifi,Kitchen,Heating,""Family/kid fri...",NaN,350.0,110.0,1,0.0


In [14]:
train_X.iloc[:5, 38:]

,minimum_nights,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,price
21300,1,flexible,f,f,35.0
8742,3,flexible,f,f,99.0
41079,1,flexible,f,f,45.0
45075,1,moderate,f,f,30.0
32280,3,strict_14_with_grace_period,f,f,150.0


In [72]:
train_X.minimum_nights.unique()

array([   1,    3,   10,    5,    2,    4,    7,   30,   90,    9,   87,
         60,    6,   14,   91,   20,  180,   31,   28,   75,   15,    8,
         19,   13,   80,   21,   12,   18,   40,   29,   25,   24,   16,
         27,  150,   50,   63,   69,   35,  168,   17,  120,  182,   85,
         49,   55,  100,   11,  112,   42,  500,   45,   58,   84,   56,
         92,   99,   62,  360,  111,  307,   70,  190,   22,   54,   59,
         89,  300,  900,   46,   64,  365,  130,  183,  250,   77,  160,
       1000,   48,  287,   95])

In [54]:
categoric_cols = ['property_type', 'bed_type', 'room_type', 'neighbourhood_cleansed']
number_cols = ['accommodates', 'bathrooms', 'bedrooms', 'beds', 'minimum_nights']

In [55]:
num_col_medians = dict()
for col in number_cols:
    num_col_medians[col] = train[col].median()

In [56]:
# encoder = OneHotEncoder(handle_unknown='ignore')
# for col in categoric_cols:
#     train[col].fillna('None', inplace=True)
# encoder.fit(train[categoric_cols]);

In [57]:
def features_transform(df):
    new_df = df.copy()
    for col in number_cols:
        new_df[col].fillna(num_col_medians[col], inplace=True)
    for col in categoric_cols:
        new_df[col].fillna('None', inplace=True)
#     df_dumm = pd.DataFrame(encoder.transform(new_df[categoric_cols]).toarray(), columns=encoder.get_feature_names())
#     new_df = pd.concat([new_df, df_dumm], axis=1)
    return new_df

In [58]:
X_train_num = features_transform(train_X)[categoric_cols + number_cols + ['price']]
X_valid_num = features_transform(valid_X)[categoric_cols + number_cols + ['price']]
X_test_num = features_transform(test)[categoric_cols + number_cols]

In [59]:
pipeline = Pipeline([('liner', StandardScaler()),
                     ('clf', LinearRegression())])
forest = RandomForestRegressor(n_estimators=150, criterion='mae', n_jobs=-1)
lgbm = LGBMRegressor(n_estimators=200)
catbst = CatBoostRegressor(loss_function='MAE')

In [60]:
X_train_num['sample_weight'] = X_train_num.price.apply(lambda s: 100 / s)
num_sample = X_train_num.sample(60000, weights=X_train_num.sample_weight, replace=True)
train_dataset = Pool(num_sample.drop(['price', 'sample_weight'], axis=1), num_sample.price, cat_features=categoric_cols)

In [139]:
cvs = cross_val_score(forest, num_sample.drop(['price', 'sample_weight'], axis=1), num_sample.price, cv=3, scoring=mape_score)

In [79]:
X_test_num.columns

Index(['property_type', 'bed_type', 'room_type', 'neighbourhood_cleansed',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'minimum_nights',
       'price'],
      dtype='object')

In [78]:
X_valid_num.columns

Index(['property_type', 'bed_type', 'room_type', 'neighbourhood_cleansed',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'minimum_nights',
       'price'],
      dtype='object')

In [62]:
catbst.fit(train_dataset)

0:	learn: 32.1909054	total: 46ms	remaining: 45.9s
1:	learn: 31.6674955	total: 72.3ms	remaining: 36.1s
2:	learn: 31.1510363	total: 93.6ms	remaining: 31.1s
3:	learn: 30.6520062	total: 116ms	remaining: 28.8s
4:	learn: 30.1665396	total: 145ms	remaining: 28.9s
5:	learn: 29.6717481	total: 166ms	remaining: 27.6s
6:	learn: 29.2203302	total: 194ms	remaining: 27.5s
7:	learn: 28.8223482	total: 219ms	remaining: 27.1s
8:	learn: 28.4381007	total: 244ms	remaining: 26.8s
9:	learn: 28.0419816	total: 281ms	remaining: 27.8s
10:	learn: 27.6922779	total: 303ms	remaining: 27.2s
11:	learn: 27.3267922	total: 327ms	remaining: 27s
12:	learn: 27.0199747	total: 346ms	remaining: 26.3s
13:	learn: 26.6795330	total: 371ms	remaining: 26.1s
14:	learn: 26.3421747	total: 393ms	remaining: 25.8s
15:	learn: 26.0375176	total: 419ms	remaining: 25.8s
16:	learn: 25.7589579	total: 444ms	remaining: 25.7s
17:	learn: 25.5126277	total: 470ms	remaining: 25.6s
18:	learn: 25.2378814	total: 497ms	remaining: 25.7s
19:	learn: 24.9797096	t

158:	learn: 17.8876174	total: 5.19s	remaining: 27.5s
159:	learn: 17.8800917	total: 5.24s	remaining: 27.5s
160:	learn: 17.8767006	total: 5.26s	remaining: 27.4s
161:	learn: 17.8710245	total: 5.28s	remaining: 27.3s
162:	learn: 17.8663552	total: 5.31s	remaining: 27.3s
163:	learn: 17.8635748	total: 5.33s	remaining: 27.2s
164:	learn: 17.8573110	total: 5.36s	remaining: 27.1s
165:	learn: 17.8540361	total: 5.38s	remaining: 27s
166:	learn: 17.8485175	total: 5.41s	remaining: 27s
167:	learn: 17.8448594	total: 5.46s	remaining: 27s
168:	learn: 17.8395449	total: 5.5s	remaining: 27s
169:	learn: 17.8373941	total: 5.52s	remaining: 26.9s
170:	learn: 17.8329651	total: 5.54s	remaining: 26.9s
171:	learn: 17.8269223	total: 5.57s	remaining: 26.8s
172:	learn: 17.8217645	total: 5.59s	remaining: 26.7s
173:	learn: 17.8205330	total: 5.62s	remaining: 26.7s
174:	learn: 17.8178768	total: 5.65s	remaining: 26.6s
175:	learn: 17.8124110	total: 5.68s	remaining: 26.6s
176:	learn: 17.8090599	total: 5.73s	remaining: 26.7s
17

320:	learn: 17.5042442	total: 11.2s	remaining: 23.6s
321:	learn: 17.4994984	total: 11.2s	remaining: 23.6s
322:	learn: 17.4989039	total: 11.3s	remaining: 23.6s
323:	learn: 17.4967663	total: 11.3s	remaining: 23.5s
324:	learn: 17.4965018	total: 11.3s	remaining: 23.5s
325:	learn: 17.4950547	total: 11.3s	remaining: 23.4s
326:	learn: 17.4945423	total: 11.3s	remaining: 23.4s
327:	learn: 17.4939035	total: 11.4s	remaining: 23.3s
328:	learn: 17.4934875	total: 11.4s	remaining: 23.3s
329:	learn: 17.4928885	total: 11.4s	remaining: 23.2s
330:	learn: 17.4898076	total: 11.5s	remaining: 23.2s
331:	learn: 17.4878722	total: 11.5s	remaining: 23.1s
332:	learn: 17.4870706	total: 11.5s	remaining: 23.1s
333:	learn: 17.4849317	total: 11.6s	remaining: 23.1s
334:	learn: 17.4818969	total: 11.6s	remaining: 23s
335:	learn: 17.4787157	total: 11.6s	remaining: 23s
336:	learn: 17.4757877	total: 11.7s	remaining: 22.9s
337:	learn: 17.4740199	total: 11.7s	remaining: 22.9s
338:	learn: 17.4733173	total: 11.7s	remaining: 22.

479:	learn: 17.3121067	total: 16.5s	remaining: 17.9s
480:	learn: 17.3111445	total: 16.6s	remaining: 17.9s
481:	learn: 17.3105479	total: 16.6s	remaining: 17.9s
482:	learn: 17.3086418	total: 16.7s	remaining: 17.9s
483:	learn: 17.3082969	total: 16.8s	remaining: 17.9s
484:	learn: 17.3074884	total: 16.9s	remaining: 17.9s
485:	learn: 17.3060307	total: 16.9s	remaining: 17.9s
486:	learn: 17.3049791	total: 16.9s	remaining: 17.8s
487:	learn: 17.3039663	total: 16.9s	remaining: 17.8s
488:	learn: 17.3027352	total: 17s	remaining: 17.7s
489:	learn: 17.3026481	total: 17s	remaining: 17.7s
490:	learn: 17.3019566	total: 17.1s	remaining: 17.7s
491:	learn: 17.3005593	total: 17.1s	remaining: 17.6s
492:	learn: 17.2990942	total: 17.1s	remaining: 17.6s
493:	learn: 17.2982010	total: 17.1s	remaining: 17.6s
494:	learn: 17.2964327	total: 17.2s	remaining: 17.5s
495:	learn: 17.2945598	total: 17.2s	remaining: 17.5s
496:	learn: 17.2911112	total: 17.2s	remaining: 17.4s
497:	learn: 17.2897019	total: 17.3s	remaining: 17.

637:	learn: 17.1812750	total: 21.7s	remaining: 12.3s
638:	learn: 17.1811287	total: 21.8s	remaining: 12.3s
639:	learn: 17.1806185	total: 21.8s	remaining: 12.3s
640:	learn: 17.1801776	total: 21.8s	remaining: 12.2s
641:	learn: 17.1798054	total: 21.9s	remaining: 12.2s
642:	learn: 17.1789878	total: 21.9s	remaining: 12.2s
643:	learn: 17.1775092	total: 22s	remaining: 12.2s
644:	learn: 17.1765612	total: 22.1s	remaining: 12.1s
645:	learn: 17.1764277	total: 22.1s	remaining: 12.1s
646:	learn: 17.1754473	total: 22.1s	remaining: 12.1s
647:	learn: 17.1737532	total: 22.2s	remaining: 12s
648:	learn: 17.1737055	total: 22.2s	remaining: 12s
649:	learn: 17.1732884	total: 22.3s	remaining: 12s
650:	learn: 17.1726331	total: 22.3s	remaining: 11.9s
651:	learn: 17.1712695	total: 22.3s	remaining: 11.9s
652:	learn: 17.1706848	total: 22.3s	remaining: 11.9s
653:	learn: 17.1705539	total: 22.4s	remaining: 11.8s
654:	learn: 17.1699292	total: 22.4s	remaining: 11.8s
655:	learn: 17.1698221	total: 22.4s	remaining: 11.8s
6

798:	learn: 17.0740696	total: 26.5s	remaining: 6.68s
799:	learn: 17.0738452	total: 26.6s	remaining: 6.64s
800:	learn: 17.0734634	total: 26.6s	remaining: 6.61s
801:	learn: 17.0725175	total: 26.6s	remaining: 6.58s
802:	learn: 17.0720194	total: 26.7s	remaining: 6.54s
803:	learn: 17.0714635	total: 26.7s	remaining: 6.51s
804:	learn: 17.0713877	total: 26.7s	remaining: 6.47s
805:	learn: 17.0711277	total: 26.8s	remaining: 6.44s
806:	learn: 17.0710834	total: 26.8s	remaining: 6.41s
807:	learn: 17.0699325	total: 26.8s	remaining: 6.37s
808:	learn: 17.0695672	total: 26.8s	remaining: 6.34s
809:	learn: 17.0694335	total: 26.9s	remaining: 6.3s
810:	learn: 17.0690339	total: 26.9s	remaining: 6.27s
811:	learn: 17.0680911	total: 26.9s	remaining: 6.24s
812:	learn: 17.0676979	total: 27s	remaining: 6.2s
813:	learn: 17.0669795	total: 27s	remaining: 6.17s
814:	learn: 17.0667448	total: 27s	remaining: 6.13s
815:	learn: 17.0663821	total: 27.1s	remaining: 6.1s
816:	learn: 17.0657352	total: 27.1s	remaining: 6.07s
81

959:	learn: 16.9861833	total: 31.8s	remaining: 1.33s
960:	learn: 16.9854577	total: 31.9s	remaining: 1.29s
961:	learn: 16.9844623	total: 31.9s	remaining: 1.26s
962:	learn: 16.9841188	total: 31.9s	remaining: 1.23s
963:	learn: 16.9840106	total: 32s	remaining: 1.19s
964:	learn: 16.9834863	total: 32s	remaining: 1.16s
965:	learn: 16.9833871	total: 32s	remaining: 1.13s
966:	learn: 16.9820163	total: 32s	remaining: 1.09s
967:	learn: 16.9815906	total: 32.1s	remaining: 1.06s
968:	learn: 16.9812182	total: 32.1s	remaining: 1.03s
969:	learn: 16.9809186	total: 32.1s	remaining: 993ms
970:	learn: 16.9807082	total: 32.1s	remaining: 960ms
971:	learn: 16.9804126	total: 32.2s	remaining: 927ms
972:	learn: 16.9799525	total: 32.2s	remaining: 893ms
973:	learn: 16.9793280	total: 32.2s	remaining: 860ms
974:	learn: 16.9782245	total: 32.2s	remaining: 827ms
975:	learn: 16.9781154	total: 32.3s	remaining: 794ms
976:	learn: 16.9780146	total: 32.3s	remaining: 761ms
977:	learn: 16.9776340	total: 32.3s	remaining: 728ms
9

In [81]:
X_valid_num['pred'] = catbst.predict(X_valid_num.drop(['price'], axis=1))

In [82]:
mape(X_valid_num.price, X_valid_num.pred)

28.48441240374772

In [83]:
X_test_num['price'] = catbst.predict(X_test_num)

In [84]:
joblib.dump(catbst, 'model.joblib')

['model.joblib']

In [37]:
model = joblib.load('model.joblib')

In [38]:
model

In [48]:
model.

[0, 1, 2, 3, 4]

In [65]:
possible = dict()
for col in categoric_cols:
    values = list(train_X[col].unique())
    possible[col] = []
    for v in values:
        possible[col].append((v, v))

In [67]:
joblib.dump(possible, 'values_dict.joblib')

['values_dict.joblib']